In [3]:
import cv2
import numpy as np
import pyautogui
import time

In [5]:
# Load the Haar cascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [ ]:
# Initialize the webcam
cap = cv2.VideoCapture(0)

# Get screen dimensions
screen_width, screen_height = pyautogui.size()

def move_mouse(x, y):
    pyautogui.moveTo(x, y)

def get_eye_center(eyes, frame_width, frame_height):
    # Assuming the first detected eye is used for cursor control
    for (ex, ey, ew, eh) in eyes:
        eye_center_x = ex + ew // 2
        eye_center_y = ey + eh // 2
        return eye_center_x, eye_center_y
    return None, None

# Variables for smoothing
previous_x, previous_y = 0, 0
smoothing_factor = 0.2

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        eye_center_x, eye_center_y = get_eye_center(eyes, frame.shape[1], frame.shape[0])
        
        if eye_center_x is not None and eye_center_y is not None:
            # Map the eye position to screen coordinates
            screen_x = int((eye_center_x + x) / frame.shape[1] * screen_width)
            screen_y = int((eye_center_y + y) / frame.shape[0] * screen_height)
            
            # Apply smoothing
            smooth_x = int(previous_x * (1 - smoothing_factor) + screen_x * smoothing_factor)
            smooth_y = int(previous_y * (1 - smoothing_factor) + screen_y * smoothing_factor)
            
            # Move the mouse to the smoothed coordinates
            move_mouse(smooth_x, smooth_y)
            
            # Update previous coordinates
            previous_x, previous_y = smooth_x, smooth_y

            # Draw rectangles around detected eyes for debugging
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

    # Display the frame for debugging
    cv2.imshow('Eye Tracking', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()